In [1]:
%%capture
!pip install fasttext

In [4]:
import pandas as pd
df = pd.read_csv("african_tales.csv")
df.head(1)

,story_id,title,url,content,language,country,tag
0,89963a16-5d35-4f59-8014-a4003a4ca840,"THE TIGER, THE RAM, AND THE JACKAL",https://mlpp.pressbooks.pub/introphil/chapter/...,Tiger was returning home from hunting on one ...,English,Africa,rania


In [4]:
from tqdm import tqdm
tqdm.pandas()

pipe_nllb = pipeline("translation", device="cpu", model="facebook/nllb-200-3.3B", src_lang="fra_Latn", tgt_lang="eng_Latn")

def translation_fr_en(text):
    return pipe_nllb(text)[0]["translation_text"]

def tales_translation(content, lang):
    if lang == "English":
        return content
    else:
        # return translation_fr_en(content)
        lst = content.split("\n")
        for i in range(len(lst)):
            lst[i] = translation_fr_en(lst[i])
        lst = "\n".join(lst)
        return lst

df["content_en"] = df.progress_apply(lambda x : tales_translation(x.content, x.language), axis=1)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 144/144 [35:51<00:00, 14.94s/it]


In [8]:
df.to_csv("africa_tales_en_nllb.csv", index=False)

In [1]:
import pandas as pd
from datasets import Dataset

df = pd.read_csv("africa_tales_en_nllb.csv")

i = 0
data = []
for row in df.to_dict(orient="records"):
    if row["title"] != "":
        description = f"A tale from {row['country']} folklore titled {row['title']}"
    else:
        description = f"A tale from {row['country']} folklore"

    dialogue = row["content_en"]
    dic = {"index" : i, "id" : i+1, "description" : description, "dialogue" : dialogue}
    i = i+1
    data.append(dic)

df = pd.DataFrame(data)
data = Dataset.from_pandas(df)

In [2]:
from huggingface_hub import login
access_token = "<your_token>"
login(token=access_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/ue75752ba5c0c7e43e2aa578129767a6/.cache/huggingface/token
Login successful


In [3]:
data.push_to_hub("<the_model_path>")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]